In [1]:
5.51 % 5

0.5099999999999998

In [3]:
def parse_direc_frame(filename: str) -> tuple[int, int]:
    direc = None
    frame = 0
    
    for info in filename.split('_'):
        if not info[1:].isnumeric():
            continue
        val = int(info[1:])
        
        if info.startswith('d'):
            direc = val
        elif info.startswith('f'):
            frame = val
    
    return direc, frame

names = [
    "bubble_f08",
    "test_d000_f001",
]

for name in names:
    print(f"{parse_direc_frame(name) = }")

info='bubble'
info='f08'
parse_direc_frame(name) = (None, 8)
info='test'
info='d000'
info='f001'
parse_direc_frame(name) = (0, 1)


In [17]:
import json

class AnimationKeyframesManager():
    def __init__(self) -> None:
        self.keyframes: dict[str, dict[str, dict]] = {}
    
    def add_animation_keyframes(self, path: str) -> None:
        model_name = path.split('/')[-1]
        
        with open(f"{path}/_keyframes.json", 'r') as injson:
            keyframes: dict[str, dict] = json.load(injson)
        
        for statename, state in keyframes["states"].items():
            if not isinstance(state["delay"], list):
                state["delay"] = [state["delay"]] * len(state["frames"])
            
            if state["type"] == "bounce": # [0, 1, 2, 3] -invert> [3, 2, 1, 0] -trim> [_, 2, 1, _] -join> [0, 1, 2, 3, 2, 1]
                state["delay"] += state["delay"][::-1][1:-1]
                state["frames"] += state["frames"][::-1][1:-1]
            state["length"] = len(state["frames"])
            
            if len(state["delay"]) != len(state["frames"]):
                print(f"Model's '{model_name}' state '{statename}' is wrong")
        
        self.keyframes[model_name] = keyframes

ak = AnimationKeyframesManager()
ak.add_animation_keyframes("./sprites/animations/bubble")
ak.keyframes["bubble"]["states"]

{'idle': {'type': 'repeat',
  'delay': [5, 5, 5, 5, 5, 5, 5, 5, 5],
  'frames': [0, 1, 2, 3, 4, 5, 6, 7, 8],
  'length': 9},
 'move': {'type': 'bounce',
  'delay': [5, 6, 10, 4, 5, 4, 10, 6],
  'frames': [0, 1, 2, 3, 4, 3, 2, 1],
  'length': 8},
 'stop': {'type': 'random',
  'delay': [10, 30, 50, 60, 20],
  'frames': [0, 3, 5, 6, 8],
  'length': 5}}

---

animations_paths=['./sprites/animations/bubble', './sprites/animations/other', './sprites/animations/test']


In [9]:


animgr = AnimationManager("./sprites/animations")

In [ ]:
import os
import arcade

animations_paths: list[str] = []
for folder in os.listdir("./sprites/animations"):
    full_path = f"./sprites/animations/{folder}"
    animations_paths.append(full_path)

print(f"{animations_paths=}")

In [1]:
class AnimationManager():
    def __init__(self, animations_folder: str) -> None:
        self.animations: dict[str, AnimationData] = {}
        
        for model in os.listdir(animations_folder):
            full_path = f"{animations_folder}/{model}"
            self.animations[model] = AnimationData(full_path)

In [ ]:



BASE_PATH = "./rotsys/sprites/animations/"
ANIM_PATHS: dict[str, dict[str, str]] = {}
for dirpath, _, filenames in os.walk(BASE_PATH):
    anim_name = dirpath.replace(BASE_PATH, "")
    if not anim_name or not filenames:
        continue
    ANIM_PATHS[anim_name] = {fn.split('.')[0]: f"{dirpath}/{fn}" for fn in filenames}

TEXTURES = {}
for name, paths in ANIM_PATHS.items():
    TEXTURES[name] = {}
    for texname, texpath in paths.items():
        key = parse_direc_frame(texname)
        texture = arcade.load_texture(texpath)
        TEXTURES[name][key] = texture
